In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 11078525
paper_name = 'chan_zheng_2000' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/chan_zheng_2000_HAP.xlsx', sheet_name='Sheet1', header=None)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 100 x 2


In [10]:
original_data.head()

,0,1
0,YDR099W,0.10
1,YAL009W,0.01
2,YCL008C,0.01
3,YCR002C,0.01
4,YDR150W,0.01


In [11]:
original_data['orf'] = original_data[0].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, 1, orf]
Index: []


In [15]:
original_data['data'] = pd.to_numeric(original_data[1], errors='coerce')

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(100, 1)

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [1]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,1
data_type,value
orf,
YAL009W,0.0100
YAL021C,0.0001
YBR200W,0.0010
YCL008C,0.0100
YCR002C,0.0100


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,1
,data_type,value
gene_id,orf,
7,YAL009W,0.0100
19,YAL021C,0.0001
394,YBR200W,0.0010
502,YCL008C,0.0100
558,YCR002C,0.0100


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            1          
data_type         value    valuez
gene_id orf                      
7       YAL009W  0.0100  0.000376
19      YAL021C  0.0001  0.000004
394     YBR200W  0.0010  0.000038
502     YCL008C  0.0100  0.000376
558     YCR002C  0.0100  0.000376

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 11078525...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.08it/s]

Updating the data_modified_on field...
